In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In this notebook, I will use a Sentence Transformer pretrained model to encode each review in the given dataset, and then cluster these encodings, after reducing their dimension using the U-Map algorithm, with the density-based HDBSCAN algorithm. Finally, I will topic model theses clusters using an approach (demonstrated in https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6) that relies on a class-based variant of TF-IDF. 

First, I'll install some packages that are not already available om Kaggle, and import everything that I need:

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!conda install hdbscan -y

In [ ]:
from sentence_transformers import SentenceTransformer
import umap
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import pickle
from joblib import Memory

Now I'll access Reviews.csv, and the data that I actually am going to use (the textual data). While to begin with I'm going to encode all of the available reviews, for the sake of this demonstration, I'm going to use only half of the available data, since my laptop was unable to handle all of the data with the necessary algorithms. The same process shown below can of course be easily extended to all of the data.

In [ ]:
df = pd.read_csv('/kaggle/input/nlp-topic-modelling/Reviews.csv')
data = df['Text']
data.head()

Now, the first real step is to encode the textual data into vectors that can be clustered according to their semantic similarity. There are a variety of pretrained Transformer Encoder models in the Sentence Transformers package that should be able to do this encoding quite well. For the task of clustering, I will use a pretrained model that was trained on paraphrase tasks, as such a model should naturally provide good results for the similarity-based task that I am attempting.

In [ ]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
embeddings = model.encode(data, show_progress_bar=True)
df_1 = df.iloc[:len(df)//2] # Half of the data
embeddings_1 = embeddings[:embeddings.shape[0] // 2, :] # Half of the embeddings

These embedded vectors have a length of 768. While vectors of such a length could be easily clustered with, e.g., the K-Means algorithm, when it comes to HDBSCAN that task is computationally far too expensive. So, before I cluster them, I'm going to reduce their dimensions to something feasible. For the task of clustering, 5 dimensions seem to work quite well in our case; I will also prepare 2 dimensional data for the sake of visualization. I will use the Uniform Manifold Approximation and Projection (U-Map) algorithm: this is an algorithm that works well both for visualization and for general non-linear dimension reduction.

I've commented the following code out, as Kaggle doesn't provide enough memory to handle it. Instead, I will load its output from .pkl files I've uploaded.

In [ ]:
"""
umap_embeddings = umap.UMAP(n_neighbors=15,
                            n_components=5,
                            min_dist=0.0,
                            metric='cosine').fit_transform(embeddings_1)

umap_2d = umap.UMAP(n_neighbors=15,
                    n_components=2,
                    min_dist=0.0,
                    metric='cosine').fit_transform(embeddings_1)
"""

with open('/kaggle/input/reviews-embeddings/umap_embeddings_15_neighbors.pkl', 
          'rb') as file:
    umap_embeddings = pickle.load(file)

with open('/kaggle/input/reviews-embeddings/umap_2d.pkl',
          'rb') as file:
    umap_2d = pickle.load(file)

With the embeddings dimensionally reduced to manageable levels, the clustering can commence. Yet while HDBSCAN generally doesn't require as much hyperparameter-tuning as DBSCAN, I will still need to find the right hyperparameters for the sort of results that I want. I want to achieve a small but not too small number of clusters, and minimize the number of outliers. I wrote the following brute-force search algorithm to find the suitable values for min_cluster_size and min_samples; it returns all of the possible options within its parameters, since the outcome I want is fuzzily defined.

In [ ]:
def hdbscan_search(cache_dir, umap_embeddings,  min_min_samples, max_min_samples, 
                   min_min_size, max_min_size):
    mem = Memory(cache_dir)
    search_results = {}
    for min_samples in range(min_min_samples, max_min_samples+1, 25):
        for min_size in range(min_min_size, max_min_size+1, 25):
            model = hdbscan.HDBSCAN(min_cluster_size=min_size,
                                    min_samples=min_samples,
                                    memory=mem).fit(umap_embeddings)
            num_of_clusters = np.max(model.labels_)+1
            outliers = np.unique(model.labels_, return_counts=True)[1][0]
            search_results[(min_size, min_samples)] = (num_of_clusters,
                                                       outliers)
    return search_results

After a few iterations of using the above algorithm, I found hyperparameters that seemed to do adequately: a min_cluster_size of 1900 with a min_samples of 400 gave me 7 clusters and only 14006 outliers out of 284227 reviews. This is therefore the clustering model that I will use: 

In [ ]:
model = hdbscan.HDBSCAN(min_cluster_size=1900,
                        min_samples=400).fit(umap_embeddings)

The model gives the following clusters, in terms of numbers:

In [ ]:
np.unique(model.labels_, return_counts=True)

To visualize the clusters in two dimensions, I will use the following algorithm:

In [ ]:
def plot_clusters(data_2d, cluster_labels):
    result = pd.DataFrame(data_2d, columns=['x', 'y'])
    result['labels'] = cluster_labels
    outliers = result.loc[result.labels == -1, :]
    clustered = result.loc[result.labels != -1, :]
    plt.figure()
    plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
    plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, 
                cmap='hsv_r')
    plt.colorbar()

plt.rcParams['figure.figsize'] = [30, 20]
plt.rcParams.update({'font.size': 18})
plot_clusters(umap_2d, model.labels_)

Now for the most interesting part of my demonstration: topic modeling. I will use an approach which I've found to usually work quite well: a class-based variant of TF-IDF. Insteading of comparing the importance of words between documents as is done in regular TF-IDF, I will compare the importance of words between "classes" of documents: that is, "super-documents", each made up of all of the reviews within a specific cluster. I will use the following algorithm to return the 10 most important words in each cluster, which will help me characterize each cluster as a specific topic of some sort. One can play around with the max_df variable, using it to ignore terms which appear in more than a certain percentage of our super-documents, but with such a small number of clusters I've found that leaving it at 100% is best.

In [ ]:
def class_tf_idf_topic_creation(df, cluster_labels, n=10, max_df=1.0):
    m = len(df)
    class_df = pd.DataFrame(df['Text'].values, columns=['text'])
    class_df['class'] = cluster_labels
    class_df['ID'] = range(len(class_df))
    class_df = class_df[class_df['class'] != -1]
    sent_per_class = class_df.groupby(['class'], 
                                      as_index=False).agg({'text': ' '.join})
    count = CountVectorizer(stop_words='english', 
                            max_df=max_df).fit(sent_per_class.text.values)
    t = count.transform(sent_per_class.text.values).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)
    words = count.get_feature_names()
    classes = list(sent_per_class['class'])
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) \
                           for j in indices[i]][::-1] \
                   for i, label in enumerate(classes)}
    return top_n_words

top_n_words = class_tf_idf_topic_creation(df_1, model.labels_)
top_n_words

The largest topic by far, cluster 6, seems to include generally positive food reviews. Clusters 0, 2 and 3 are also comparatively large: cluster 0 seems to include pet food reviews, cluster 2 seems to include tea reviews, and cluster 3 seems to include coffee reviews. The rest of the clusters are somewhat small, but definitely seem to have clear topics: cluster 1 is composed of popcorn and theater related reviews, cluster 4 is composed of meat related, cluster 5 is composed of chips and potato related reviews, and cluster 7 is composed of plant and garden related reviews.